In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
sys.path.append('../')

from dcase_models.data.datasets import URBAN_SED
from dcase_models.model.models import MLP, SB_CNN_SED

# Features
from dcase_models.data.features import MelSpectrogram

from attprotos.layers import PrototypeReconstruction
from attprotos.losses import prototype_loss
from attprotos.model import AttProtos
from attprotos.features import Openl3


from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.scaler import Scaler
from dcase_models.util.files import load_json, load_pickle
from dcase_models.util.files import mkdir_if_not_exists, save_pickle
from dcase_models.util.data import evaluation_setup
from dcase_models.model.container import KerasModelContainer

from keras.layers import Layer

import matplotlib.pyplot as plt

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dataset_name = "URBAN_SED"
models_path = '../experiments'
dataset_path = './'
fold_name = "test"


model_folder = os.path.join(models_path, dataset_name, "AttProtos")
parameters_file = os.path.join(model_folder, 'config.json')
params = load_json(parameters_file)

params_features = params['features']["MelSpectrogram"]
if params_features['pad_mode'] == 'none':
    params_features['pad_mode'] = None

params_dataset = params['datasets'][dataset_name]
params_model = params['models']["AttProtos"]


# Get and init dataset class
dataset_path = os.path.join(dataset_path, params_dataset['dataset_path'])
dataset = URBAN_SED(dataset_path)

# Get and init feature class
features = MelSpectrogram(**params_features)

print('Features shape: ', features.get_shape(10.0))

if not features.check_if_extracted(dataset):
    print('Extracting features ...')
    features.extract(dataset)
    print('Done!')

Features shape:  (10, 20, 128)


In [ ]:
exp_folder = os.path.join(model_folder, fold_name)

# Load scaler
scaler_file = os.path.join(model_folder, fold_name + '_1', 'scaler.pickle')
scaler = load_pickle(scaler_file)

# Init data generator
data_gen_test = DataGenerator(
    dataset, features, folds=[fold_name],
    batch_size=params['train']['batch_size'],
    shuffle=False, train=False, scaler=scaler
)

metrics = ['sed']

features_shape = features.get_shape()
n_frames_cnn = features_shape[1]
n_freq_cnn = features_shape[2]
n_classes = len(dataset.label_list)

model_container = AttProtos(
    model=None, model_path=None, n_classes=n_classes,
    n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
    metrics=metrics,
    **params_model['model_arguments']
)

F1 = np.zeros(10)
ER = np.zeros(10)

for j in range(1, 11):
    exp_folder = os.path.join(model_folder, fold_name + '_' + str(j))

    model_container.load_model_weights(exp_folder)
    results = model_container.evaluate(data_gen_test, label_list=dataset.label_list)
    results = results['sed'].results()
    F1[j-1] = results['overall']['f_measure']['f_measure']
    ER[j-1] = results['overall']['error_rate']['error_rate']
    
    print(j, F1[j-1], ER[j-1])

Instructions for updating:
Colocations handled automatically by placer.
1 0.5786741610183502 0.5738399141118932
2 0.5871659574468086 0.5653107479422641
3 0.5928188464662876 0.5681736848383634


In [ ]:
print(np.mean(F1), np.mean(ER))
print(np.std(F1), np.std(ER))

In [ ]:
model_folder = os.path.join(models_path, dataset_name, "SB_CNN_SED")
parameters_file = os.path.join(model_folder, 'config.json')
params = load_json(parameters_file)

params_features = params['features']["MelSpectrogram"]
if params_features['pad_mode'] == 'none':
    params_features['pad_mode'] = None

params_dataset = params['datasets'][dataset_name]
params_model = params['models']["SB_CNN_SED"]


# Get and init feature class
features = MelSpectrogram(**params_features)

print('Features shape: ', features.get_shape(10.0))

if not features.check_if_extracted(dataset):
    print('Extracting features ...')
    features.extract(dataset)
    print('Done!')

In [ ]:
exp_folder = os.path.join(model_folder, fold_name)

# Load scaler
scaler_file = os.path.join(model_folder, fold_name + '_1', 'scaler.pickle')
scaler = load_pickle(scaler_file)

# Init data generator
data_gen_test = DataGenerator(
    dataset, features, folds=[fold_name],
    batch_size=params['train']['batch_size'],
    shuffle=False, train=False, scaler=scaler
)

features_shape = features.get_shape()
n_frames_cnn = features_shape[1]
n_freq_cnn = features_shape[2]
n_classes = len(dataset.label_list)

model_container = SB_CNN_SED(
    model=None, model_path=None, n_classes=n_classes,
    n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
    metrics=metrics,
    **params_model['model_arguments']
)

F1_CNN = np.zeros(10)
ER_CNN = np.zeros(10)

for j in range(1, 11):
    exp_folder = os.path.join(model_folder, fold_name + '_' + str(j))

    model_container.load_model_weights(exp_folder)
    results = model_container.evaluate(data_gen_test, label_list=dataset.label_list)
    results = results['sed'].results()
    F1_CNN[j-1] = results['overall']['f_measure']['f_measure']
    ER_CNN[j-1] = results['overall']['error_rate']['error_rate']
    
    print(j, F1_CNN[j-1], ER_CNN[j-1])

In [ ]:
print(np.mean(F1_CNN), np.mean(ER_CNN))
print(np.std(F1_CNN), np.std(ER_CNN))

In [ ]:
model_folder = os.path.join(models_path, dataset_name, "MLP")
parameters_file = os.path.join(model_folder, 'config.json')
params = load_json(parameters_file)

params_features = params['features']["Openl3"]

params_dataset = params['datasets'][dataset_name]
params_model = params['models']["MLP"]


# Get and init feature class
features = Openl3(**params_features)

print('Features shape: ', features.get_shape(10.0))

if not features.check_if_extracted(dataset):
    print('Extracting features ...')
    features.extract(dataset)
    print('Done!')

In [ ]:
exp_folder = os.path.join(model_folder, fold_name + '_1')

# Load scaler
scaler_file = os.path.join(model_folder, fold_name + '_1', 'scaler.pickle')
scaler = load_pickle(scaler_file)

# Init data generator
data_gen_test = DataGenerator(
    dataset, features, folds=[fold_name],
    batch_size=params['train']['batch_size'],
    shuffle=False, train=False, scaler=scaler
)

features_shape = features.get_shape()
#n_frames_cnn = features_shape[1]
#n_freq_cnn = features_shape[2]
n_classes = len(dataset.label_list)

model_container = MLP(
    model=None, model_path=exp_folder,
    metrics=metrics,
)

F1_MLP = np.zeros(10)
ER_MLP = np.zeros(10)

for j in range(1, 11):
    exp_folder = os.path.join(model_folder, fold_name + '_' + str(j))

    model_container.load_model_weights(exp_folder)
    results = model_container.evaluate(data_gen_test, label_list=dataset.label_list)
    results = results['sed'].results()
    F1_MLP[j-1] = results['overall']['f_measure']['f_measure']
    ER_MLP[j-1] = results['overall']['error_rate']['error_rate']
    
    print(j, F1_MLP[j-1], ER_MLP[j-1])

In [ ]:
print(np.mean(F1_MLP), np.mean(ER_MLP))
print(np.std(F1_MLP), np.std(ER_MLP))


print("{:f} +- {:f}".format(np.mean(F1*100), np.std(F1*100)))
print("{:f} +- {:f}".format(np.mean(F1_CNN*100), np.std(F1_CNN*100)))
print("{:f} +- {:f}".format(np.mean(F1_MLP*100), np.std(F1_MLP*100)))

print("{:f} +- {:f}".format(np.mean(ER), np.std(ER)))
print("{:f} +- {:f}".format(np.mean(ER_CNN), np.std(ER_CNN)))
print("{:f} +- {:f}".format(np.mean(ER_MLP), np.std(ER_MLP)))

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
c = 'green'
ax1.boxplot([F1*100, F1_CNN*100, F1_MLP*100], positions=[1,2,3], patch_artist=True,
            boxprops=dict(facecolor=c, color=c),
            capprops=dict(color=c),
            whiskerprops=dict(color=c),
            flierprops=dict(color=c, markeredgecolor=c),
            medianprops=dict(color=c),)
c = 'blue'
ax2.boxplot([ER, ER_CNN, ER_MLP], positions=[1.3,2.3,3.3], patch_artist=True,
            boxprops=dict(facecolor=c, color=c),
            capprops=dict(color=c),
            whiskerprops=dict(color=c),
            flierprops=dict(color=c, markeredgecolor=c),
            medianprops=dict(color=c),)

#ax1.set_xticks([1,2,3], ['Ours', 'CNN', 'Openl3'])
#ax2.set_xticks([1.2,2.2,3.2], ['Ours', 'CNN', 'Openl3'])

ax1.set_xlabel('X data')
ax1.set_ylabel('F1 (%)', color='g')
ax2.set_ylabel('ER', color='b')
ax2.set_ylim(ax2.get_ylim()[::-1])